In [1]:
import sys
import os
sys.path.insert(0, '../verificator')

In [2]:
import numpy as np

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
from verificator.tpe_predictor import TPEPredictor

Using TensorFlow backend.


In [5]:
predictor = TPEPredictor(2048, 512)

../verificator/verificator/tpe_model/tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="linear", weights=[array([[ ..., use_bias=False, input_dim=2048)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
../verificator/verificator/tpe_model/tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
/home/shareduser/.local/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
../verificator/verificator/tpe_model/tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outp

## Загружаем фичи для индексирования

In [7]:
base_loading_path = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/cam_1')

In [8]:
labels = np.load(os.path.join(base_loading_path, 'cam_1_inception_labels.npy'))
features = np.load(os.path.join(base_loading_path, 'cam_1_inception_features.npy'))

In [10]:
features.shape

(19447, 2048)

## Создаем tpe-фичи для индексирования

In [11]:
predictor.load_weights('/home/shareduser/d_test/data/models/final_models/weights/inception/cam_1/cam_1_best.h5')

In [12]:
features = predictor.predict(features)

In [13]:
def make_map_from_features(labels):
    index_dict = {}
    for i, label in enumerate(labels):
        index_dict[i] = label
    return index_dict

### index_dict - словарь для хранения соотвествия индекса ветора и номера класса

In [14]:
index_dict = make_map_from_features(labels)

In [15]:
from annoy import AnnoyIndex

In [16]:
t = AnnoyIndex(512)

In [17]:
features.shape

(19447, 512)

In [18]:
for i, feature in enumerate(features):
    t.add_item(i, feature)

In [19]:
t.build(10)

True

In [20]:
index_saving_path = os.path.expanduser('~/d_test/data/models/final_models/indexes')

In [21]:
t.save(os.path.join(index_saving_path, 'cam_1_index.ann'))

True

In [22]:
np.save(os.path.join(index_saving_path, 'cam_1_index_dict.npy'), index_dict)

## Создаем тестовые tpe-фичи

In [23]:
test_loading_path = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/test/cam_1')

In [24]:
test_labels = np.load(os.path.join(test_loading_path, 'cam_1_test_labels.npy'))
test_features = np.load(os.path.join(test_loading_path, 'cam_1_test_features.npy'))

In [25]:
test_tpe_features = predictor.predict(test_features)

In [26]:
test_tpe_features.shape

(2605, 512)

In [30]:
v = test_tpe_features[0]
l = labels[0]

In [34]:
indices = t.get_nns_by_vector(v, 10)

In [35]:
probable_neighbours_classes = []
for ind in indices:
    probable_neighbours_classes.append(index_dict[ind])

In [36]:
probable_neighbours_classes

[355, 355, 355, 386, 355, 355, 534, 355, 355, 460]

In [38]:
indices

[16, 19, 5, 690, 6, 0, 13733, 24, 4, 15790]

In [39]:
v_gt = features[16]

In [42]:
for i in indices:
    v_gt = features[i]
    print(v@v_gt.T, index_dict[i])

0.621246 355
0.617971 355
0.593282 355
0.535644 386
0.528389 355
0.527855 355
0.505349 534
0.500719 355
0.491118 355
0.482095 460


### Загрузим фичи персон, которых почти наверняка не было

In [43]:
wrong_loading_path = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/cam_2')

In [44]:
wrong_features = np.load(os.path.join(wrong_loading_path, 'cam_2_inception_features.npy'))
wrong_labels = np.load(os.path.join(wrong_loading_path, 'cam_2_inception_labels.npy'))

In [45]:
wrong_tpe_features = predictor.predict(wrong_features)

In [136]:
all_scores = wrong_tpe_features @ features.T

In [138]:
all_scores.shape

(3755, 19447)

In [166]:
for label in unique_unknown_labels: #для каждой неизвестной персоны
    pers_scores = all_scores[np.where(wrong_labels == label)]
    mean_scores = np.mean(pers_scores, axis=0)
    best_inds = np.argpartition(mean_scores, -5)[-5:]
    best_inds = best_inds[np.argsort(mean_scores[best_inds])]
    #print(best_inds)
    best_labels = labels[best_inds]
    best_scores = mean_scores[best_inds]
    print(label, best_labels, best_scores)
    
    

567 [374  50 374  85   6] [ 0.43972301  0.44163784  0.44384909  0.44549942  0.4524579 ]
568 [399 407  22 407 374] [ 0.39954966  0.4001615   0.40191445  0.42572194  0.4298138 ]
569 [108 108 108  85 190] [ 0.48127094  0.48848754  0.49510342  0.4996621   0.50312746]
570 [108 108 334 318 108] [ 0.41359186  0.41938943  0.43093932  0.45079774  0.45277977]
571 [ 20 278 318 278 278] [ 0.36459711  0.36561713  0.37459871  0.37571186  0.37688288]
572 [526  98 103 526  97] [ 0.34323505  0.34574729  0.3483066   0.34844002  0.34905514]
573 [108 481 190 318 481] [ 0.4897646   0.49686462  0.50314808  0.50925702  0.51577866]
574 [167 318 515 318 318] [ 0.37891829  0.37991229  0.39599693  0.40330312  0.41001195]
575 [526 131 131 131 526] [ 0.35971785  0.36448953  0.36548832  0.36598003  0.36966407]
576 [391 324 324 407 183] [ 0.41349629  0.41455194  0.4190881   0.4290024   0.44160852]
577 [161 481 190 190 190] [ 0.47554263  0.47581279  0.48209336  0.4823688   0.5071364 ]
578 [111  95 145  95  95] [ 0.47

In [148]:
np.array([[1,2], [3,4,5]])

array([list([1, 2]), list([3, 4, 5])], dtype=object)

#### определяем размер поиска

In [120]:
unique_known_labels = np.unique(labels)

In [151]:
max_img_per_pers = 0
for label in unique_known_labels:
    n_pers = len(labels[np.where(labels==label)])
    if n_pers > max_img_per_pers:
        max_img_per_pers = n_pers

In [152]:
max_img_per_pers

152

## Ищем top-n персон, похожих на каждую персону в imposter-выборке

In [133]:
unique_unknown_labels = np.unique(wrong_labels)

In [154]:
for label in unique_unknown_labels:
    pers_tpe_features = wrong_tpe_features[np.where(wrong_labels == label)] # выбрали фичи каждой персоны
    pers_indices = np.array([])
    for feature in pers_tpe_features:
        indices = t.get_nns_by_vector(feature, 5*max_img_per_pers)
        pers_indices = np.append(pers_indices, indices)
        

In [131]:
wrong_tpe_features[np.nonzero(wrong_labels==wrong_labels[0])].shape

(34, 512)

In [46]:
wv = wrong_tpe_features[0]

In [47]:
w_indices = t.get_nns_by_vector(wv, 10)

In [49]:
for i in w_indices:
    v_gt = features[i]
    print(wv@v_gt.T, index_dict[i])

0.686537 283
0.623115 541
0.622217 283
0.602612 283
0.602293 126
0.586456 283
0.581604 126
0.578093 126
0.575806 283
0.572738 126


In [64]:
features[0].shape

(512,)

In [115]:
scores = (wrong_tpe_features@features.T<0.62)

In [116]:
anss=np.all(scores, axis=1).astype('int')

In [117]:
len(wrong_labels)

3755

In [118]:
np.sum(anss)

3070

In [119]:
np.sum(anss)/len(wrong_labels)

0.81757656458055927

In [58]:
len(labels)

19447

In [59]:
ans

[355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 194,
 194,
 35,
 386,
 386,
 386,
 386,
 386,
 386,
 353,
 353,
 353,
 353,
 353,
 353,
 353,
 201,
 201,
 53,
 334,
 334,
 148,
 148,
 209,
 209,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 281,
 468,
 468,
 138,
 138,
 138,
 138,
 138,
 138,
 138,
 138,
 138,
 138,
 138,
 275,
 275,
 275,
 275,
 275,
 275,
 275,
 275,
 275,
 275,
 275,
 318,
 402,
 402,
 402,
 411,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 351,
 498,
 498,
 47,
 47,
 18,
 458,
 471,
 471,
 471,
 471,
 471,
 471,
 471,
 542,
 542,
 542,
 542,
 542,
 557,
 557,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 83,
 302,
 456,
 406,
 406,
 406,
 406,
 406,
 406,
 406,
 487,
 487,
 487,
 197,


In [60]:
653-29

624

In [61]:
19447-624

18823

In [62]:
18823/19447

0.9679127886049262